# **WRF-Python Tutorial - 2025**


## *Douglas Lima de Bem (UFSM/URCA)*


#### **Topics Covered Today**
*   Overview
*   Getting started
*   Use the WRF-Python functions
*   Plotting with WRF-Python using Cartopy and Matplotlib

****

### **What is WRF-Python?**

WRF-Python it is a python library that provide a collection of diagnostic and interpolation routines for use with output from the Weather Research and Forecasting (WRF-ARW) Model.

*   Contains over 30 diagnostic calculations (Cloud Fraction, Updraft Helicity, CAPE, etc.)
*   Several interpolation routines
*   Utilities to help with plotting via cartopy, basemap, or PyNGL


### **Informations and updates of WRF-Python**

To find more information about the utilities and ease of use of WRF-Python, you can search in:



*  [WRF-Python Read the Docs](https://wrf-python.readthedocs.io/en/latest/new.html)
*  [Github Repository](https://github.com/NCAR/wrf-python)
*  [WRF-Python Google Group](https://groups.google.com/a/ucar.edu/g/wrfpython-talk?pli=1)

****

### **Getting Started**
This library, despite being very useful, has not received updates since 2022. There have already been some discussions in the group about the reasons why there have been no updates. Due to this absence, it will have some dependencies that must be taken into account before its use (other libraries that must be used). For this reason, it is recommended to use an environment which contains the necessary versions of the libraries.




*   The versions for the packages being used today will be:

    - wrf-python==1.3.4.1         
    - xarray==2024.9.0
    - netCDF4==1.7.1.post2
    - numpy==1.24.1
    - matplotlib==3.8.4
    - cartopy==0.24.1


**Note**: if your Jupyter Notebook crashes at any point, make sure to re-run the following code cell again before any others.